In [2]:
import cv2
import Levenshtein
import optuna
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn.metrics
import ipywidgets
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
tqdm.pandas()

/opt/miniconda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
%load_ext autoreload
%autoreload 2
from nncomp_molecule import constants

In [4]:
train_df = pd.read_pickle("/work/input/kfujikawa/bms-preprocess-v2/train.pkl").set_index("image_id")
train_df.head()

,InChI,image_path,InChI_length,w,h,w/h,w*h,is_flipped
image_id,,,,,,,,
4435736fd10b,InChI=1S/C65H110O6/c1-4-7-10-13-16-19-22-25-28...,/work/input/bms-molecular-translation/train/4/...,403,1472,788,1.868020,1159936,False
8c0e35ce3f1f,InChI=1S/C60H98O6/c1-4-7-10-13-16-19-22-25-27-...,/work/input/bms-molecular-translation/train/8/...,398,939,309,3.038835,290151,False
934593ad3cae,InChI=1S/C63H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/9/...,397,826,661,1.249622,545986,False
c963808e309d,InChI=1S/C59H92O6/c1-4-7-10-13-16-19-22-25-28-...,/work/input/bms-molecular-translation/train/c/...,393,1268,883,1.436014,1119644,False
0aa425d5f5ac,InChI=1S/C62H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/0/...,393,918,543,1.690608,498474,False


In [9]:
VALID_CSVs = [
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=16.csv",
#     constants.OUTPUTDIR / "1113_swin_large_bert_384" / "candidate_0521_1113_swin_large_bert_384.csv",
#     constants.OUTPUTDIR / "1106_swin_bert_384" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "1106_swin_bert_384" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "9004_1102+1105" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9005_1102+1105+1106" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "9005_1102+1105+1106" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "9007_1109+1113" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9007_1109+1113" / "valid_beam=4.csv",
    
#     constants.OUTPUTDIR / "v30" / "valid_pred.csv",
]
COLUMNS = [
    "image_id",
    "InChI",
    "normed_InChI",
    "is_valid",
    "normed_score",
    "InChI_GT",
    "levenshtein",
]
valid_df = pd.concat([
    pd.read_csv(filename, usecols=COLUMNS).assign(model=filename.parent.name)
    for filename in tqdm(VALID_CSVs)
], ignore_index=True)
display(valid_df.groupby("model").agg(**{
    "count": ("image_id", "count"),
    "levenshtein": ("levenshtein", "mean"),
    "is_valid": ("is_valid", "mean"),
    "normed_score": ("normed_score", "mean")
}))

,count,levenshtein,is_valid,normed_score
model,,,,
1113_swin_large_bert_384,1212095,8.214288,0.561736,0.071950
9005_1102+1105+1106,242419,0.898923,0.985942,0.002378
9006_1103+1106+1109,1212095,13.574322,0.614990,0.072085
9007_1109+1113,1212095,12.033737,0.599610,0.072635


In [10]:
sort_keys = dict(
    image_id=True,
    is_valid=False,
    normed_score=True,
)
valid_ensembled_df = valid_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby(["image_id", "model"]).first()
valid_ensembled_df.groupby("model").levenshtein.mean()

model
1113_swin_large_bert_384    0.868298
9005_1102+1105+1106         0.898923
9006_1103+1106+1109         0.787067
9007_1109+1113              0.759503
Name: levenshtein, dtype: float64

In [11]:
base_score = valid_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby("image_id").first().levenshtein.mean()
lower_score = valid_df.groupby("image_id").levenshtein.min().mean()

print(f"Levenshtein: {base_score}")
print(f"Levenshtein (lower): {lower_score}")

Levenshtein: 0.735825162219133
Levenshtein (lower): 0.1739632619555398


In [7]:
voting_df = valid_df.groupby(["image_id", "normed_InChI"]).model.count()
valid_df["votes"] = valid_df[["image_id", "normed_InChI"]].progress_apply(
    lambda x: voting_df.loc[x.image_id, x.normed_InChI],
    axis=1
)

In [8]:
sort_keys = dict(
    image_id=True,
    is_valid=False,
#     votes=False,
    normed_score=True,
)
base_score = valid_ensembled_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby("image_id").first().levenshtein.mean()
lower_score = valid_df.groupby("image_id").levenshtein.min().mean()

print(f"Levenshtein: {base_score}")
print(f"Levenshtein (lower): {lower_score}")

Levenshtein: 0.7396903708042687
Levenshtein (lower): 0.16620809425003816


In [9]:
invalid_df = valid_df[valid_df.normed_InChI.apply(lambda x: "/q" in x)]

In [10]:
sample = invalid_df.sort_values(["votes", "image_id"]).iloc[-1]